In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/test.csv')

In [ ]:
from PIL import Image

In [ ]:
import sys
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm as tqdm
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
import random

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from sklearn.metrics import roc_auc_score
import albumentations

import pydicom
import os, os.path as osp


from scipy.ndimage.interpolation import zoom
from tqdm import tqdm
from glob import glob

transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.43216, 0.394666, 0.37645],
                            std=[0.22803, 0.22145, 0.216989])
])
Transform = transforms.Compose([
      transforms.ToTensor()
])
trans1 = transforms.ToPILImage()

In [ ]:
trans1 = transforms.ToPILImage()

In [ ]:
def load_dicom_array(dicom_files):
    dicom_files = glob(osp.join(dicom_files, '*.dcm'))
    dicoms = [pydicom.dcmread(d) for d in dicom_files]
    # Assume all images are axial
    z_pos = [float(d.ImagePositionPatient[-1]) for d in dicoms]
    return np.asarray(dicom_files)[np.argsort(z_pos)]

def edit_filenames(files):
    dicoms = [f"{ind:04d}_{f.split('/')[-1]}" for ind,f in enumerate(files)]
    series = ['/'.join(f.split('/')[-3:-1]) for f in files]
    return [osp.join(s,d) for s,d in zip(series, dicoms)]


class Lungs(Dataset):
    def __init__(self, dicom_folders):
        self.dicom_folders = dicom_folders
    def __len__(self): return len(self.dicom_folders)
    def get(self, i):
        return load_dicom_array(self.dicom_folders[i])
    def __getitem__(self, i):
        try:
            return self.get(i)
        except Exception as e:
            print(e)
            return None


MAX_LENGTH = 256.

df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/test.csv')
dicom_folders = list(('../input/rsna-str-pulmonary-embolism-detection/test' + df.StudyInstanceUID + '/'+ df.SeriesInstanceUID + '/').unique())
dset = Lungs(dicom_folders)
loader = DataLoader(dset, batch_size=1, shuffle=False, num_workers=0, collate_fn=lambda x: x)
df['path'] = df['SOPInstanceUID']
df['position'] = df['StudyInstanceUID']
positions = dict(zip(df['SOPInstanceUID'], df.index))

for data in tqdm(loader, total=len(loader)):
    data = data[0]
    if type(data) == type(None):
        continue
    try:
        files = data
        files = edit_filenames(files)
        for i in range(len(files)):
            SOP = files[i].split('/')[2].split('.')[0].split('_')[1]
            idx = positions[SOP]
            df.at[idx, 'path'] = data[i]
            df.at[idx, 'position'] = files[i]
    except Exception as e:
        print(e)
df = df.sort_values('position').reset_index().drop(['index'], axis = 1)

In [ ]:
def reduce_resolution_CT(inpt_exam, outpt_size):
    if len(inpt_exam) < outpt_size:
        return(inpt_exam)
    result = pd.DataFrame(columns = inpt_exam.columns)
    temp = inpt_exam.reset_index().drop(['index'], axis = 1)
    fact = int((len(inpt_exam)*10/outpt_size))
    rm_count = fact - 10
    #remove rm_count element from every fact batch
    step = int(len(inpt_exam)/fact)
    for i in range(step):
        batch_t = temp[temp.index <= (i + 1)*fact]
        batch = batch_t[batch_t.index > i*fact]
        #rm_batch = batch[batch.pe_present_on_image == 0]
        rm_batch = batch
        if len(rm_batch) >= rm_count:
            rm_idx = random.sample(list(rm_batch.index), rm_count)
            batch.drop(rm_idx, axis = 0, inplace = True)
            #print(rm_idx)
        else:
            batch.drop(rm_batch.index, axis = 0, inplace = True)
            rm_idx = random.sample(list(batch.index), rm_count - len(rm_batch))
            batch.drop(rm_idx, axis = 0, inplace = True)
            #print(rm_idx)
        result = pd.concat([result, batch]).reset_index().drop(['index'], axis = 1)
    if len(result) == outpt_size -1:
        result = pd.concat([result, result[len(result)-1:]]).reset_index().drop(['index'], axis = 1)
    return(result)

In [ ]:
def distinct_slices(inpt_exam, window_size):
    temp = inpt_exam.copy().reset_index().drop(['index'], axis=1)
    temp['slice'] = inpt_exam['SOPInstanceUID']
    step_num = int(len(inpt_exam)/window_size)
    idx = 0
    for i in range(step_num):
        instance = temp['SOPInstanceUID'][i*window_size]
        for j in range(window_size):
            idx = i*window_size + j
            temp['slice'][idx] = instance
    if idx < len(inpt_exam)-1:
        idx += 1
        instance = temp['SOPInstanceUID'][idx]
        temp['slice'][idx] = instance
        for j in range(len(inpt_exam) - idx):
            idx += 1
            temp['slice'][idx] = instance
    return(temp)

In [ ]:
def table_3D(table, max_resol, window_size):
    Exams = list(table['StudyInstanceUID'].unique())
    #reduce resolution
    l_res = pd.DataFrame(columns = table.columns)
    l_res['slice'] = l_res['SOPInstanceUID']
    for i, Exam in tqdm(enumerate(Exams)):
        temp = table[table.StudyInstanceUID == Exam]
        temp = reduce_resolution_CT(temp, max_resol)
        #distinguish slices
        temp = distinct_slices(temp, window_size)
        l_res = pd.concat([l_res, temp])
    l_res = l_res.reset_index().drop(['index'], axis = 1)
    return(l_res)

In [ ]:
tij = table_3D(df, max_resol = 192, window_size = 48)

In [ ]:
def load_detect(net):
    big_detect_net = net
    big_detect_net.load_state_dict(torch.load('../input/mymodels/best_bigdetect.pth'))
    big_detect_net = big_detect_net.eval()
    #fscore 50% (good recall)
    return(big_detect_net.cuda())
def load_right(net):
    right_det_net = net
    right_det_net.load_state_dict(torch.load('../input/mymodels/PEright_45_49.pth'))
    right_det_net = right_det_net.eval()
    #detection: 45% + right: 49%
    return(right_det_net.cuda())
def load_left(net):
    left_net = net
    left_net.load_state_dict(torch.load('../input/mymodels/best_left.pth'))
    left_net = left_net.eval()
    #fscore 48% acc 91%
    return(left_net.cuda())
def load_central(net):
    central_net = net
    central_net.load_state_dict(torch.load('../input/mymodels/best_central.pth'))
    central_net = central_net.eval()
    #fscore 57%
    return(central_net.cuda())
models = {'big_detect_net': load_detect, 'right_det_net': load_right,
          'left_net': load_left, 'central_net': load_central}
modelslist = ['big_detect_net', 'right_det_net', 'left_net', 'central_net']
C = {'big_detect_net': ['negative_detect', 'positive_detect'], 'right_det_net' : ['right_detect', 'right'],
    'left_net': ['negative_left', 'positive_left'], 'central_net': ['negative_central', 'positive_central']}

In [ ]:
def empty_pipeline(tab):
    eval_tab = pd.DataFrame(columns = ['Exam', 'slice', 'negative_detect', 'positive_detect', 
                                      'right_detect', 'right', 'negative_left', 'positive_left',
                                      'negative_central', 'positive_central', 'negative_chronic','chronic',
                                     'acute', 'qa_motion', 'qa_contrast', 'flow_artifact', 'true_filling_defect_not_pe'])
    Exams = tab['StudyInstanceUID'].dropna().unique()
    for i, exam in tqdm(enumerate(Exams)):
        B1 = [0]*17
        B2 = [0]*17
        B3 = [0]*17
        B4 = [0]*17
        ex = tab[tab.StudyInstanceUID == exam].reset_index().drop(['index'], axis = 1)
        slices = ex['slice'].dropna().unique()
        B1[0] = exam
        B2[0] = exam
        B3[0] = exam
        B4[0] = exam
        B1[1] = slices[0]
        B2[1] = slices[1]
        try:
            B3[1] = slices[2]
        except:
            B3[1] = 'noslice'
        try:
            B4[1] = slices[3]
        except:
            B4[1] = 'noslice'
        
        B1 = pd.DataFrame([B1], columns= eval_tab.columns)
        B2 = pd.DataFrame([B2], columns= eval_tab.columns)
        B3 = pd.DataFrame([B3], columns= eval_tab.columns)
        B4 = pd.DataFrame([B4], columns= eval_tab.columns)
        
        eval_tab = pd.concat([eval_tab, B1, B2, B3, B4])
    return(eval_tab.reset_index().drop(['index'], axis = 1))

In [ ]:
eval_tab = empty_pipeline(tij)
for col in eval_tab.columns[2:14]:
    eval_tab[col] = pd.to_numeric(eval_tab[col], downcast="float")

In [ ]:
device = torch.device('cuda')
def load_dicom_array(dicom_files):
    dicoms = [pydicom.dcmread(d) for d in dicom_files]
    M = float(dicoms[0].RescaleSlope)
    B = float(dicoms[0].RescaleIntercept)
    # Assume all images are axial
    z_pos = [float(d.ImagePositionPatient[-1]) for d in dicoms]
    dicoms = np.asarray([d.pixel_array for d in dicoms])
    dicoms = dicoms[np.argsort(z_pos)]
    dicoms = dicoms * M
    dicoms = dicoms + B
    return dicoms, np.asarray(dicom_files)[np.argsort(z_pos)]

def window(img, WL=50, WW=350):
    upper, lower = WL+WW//2, WL-WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    X = (X*255.0).astype('uint8')
    return X

def read_image(dicom_files, image_size=256):
    image, files = load_dicom_array(dicom_files)
    # Windows from https://pubs.rsna.org/doi/pdf/10.1148/rg.245045008
    image_lung = np.expand_dims(window(image, WL=-600, WW=1500), axis=3)
    image_mediastinal = np.expand_dims(window(image, WL=40, WW=400), axis=3)
    image_pe_specific = np.expand_dims(window(image, WL=100, WW=700), axis=3)
    image = np.concatenate([image_lung, image_pe_specific, image_mediastinal], axis=3)
    rat = image_size / np.max(image.shape[1:])
    image = zoom(image, [1.,rat,rat, 1.], prefilter=False, order=1)
    image = Transform(image[0])
    return image

def read_dicom(dicom_files, image_size=256):
    image, files = load_dicom_array(dicom_files)
    # Windows from https://pubs.rsna.org/doi/pdf/10.1148/rg.245045008
    image_lung = np.expand_dims(window(image, WL=-600, WW=1500), axis=3)
    image_mediastinal = np.expand_dims(window(image, WL=40, WW=400), axis=3)
    image_pe_specific = np.expand_dims(window(image, WL=100, WW=700), axis=3)
    image = np.concatenate([image_lung, image_pe_specific, image_mediastinal], axis=3)
    rat = image_size / np.max(image.shape[1:])
    image = zoom(image, [1., rat,rat, 1.], prefilter=False, order=1)
    image = transform(image[0]).unsqueeze(0)
    output = torch.zeros((3,48,256,256))
    output[:,:image.size(1), :, :] = image
    return output

In [ ]:
#read_image(['../input/rsna-str-pulmonary-embolism-detection/test/00268ff88746/75d23269adbd/012c12fe09c3.dcm']).shape

In [ ]:
class PE_dataset_pipeline(torch.utils.data.Dataset):
    def __init__(self, eval_tab, tab, window = 48):
        self.slices = list(eval_tab['slice'].unique())
        self.slices.remove('noslice')
        self.window = window
        self.Table = eval_tab
        self.tab = tab
            
    def __getitem__(self, index):
        mini_batch = self.Table[self.Table.slice == self.slices[index]]
        mb = self.tab[self.tab.slice == self.slices[index]]
        Input = read_dicom(mb['path'])
        return([Input, mini_batch.index[0]])
    def __len__(self):
        return(len(self.slices))

In [ ]:
import math
import torch
import torch.nn as nn

class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block.

    Based on the paper:
    "Squeeze-and-Excitation Networks"
    by Jie Hu, Li Shen, Gang Sun
    (https://arxiv.org/abs/1709.01507).
    """

    def __init__(self, num_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool3d(1)
        self.excite = nn.Sequential(nn.Linear(num_channels, num_channels // reduction),
                                    nn.LeakyReLU(inplace=True),
                                    nn.Linear(num_channels // reduction, num_channels),
                                    nn.Sigmoid())

    def forward(self, x):
        num_channels = x.size(1)

        # Squeeze
        z = self.squeeze(x)
        z = z.view(-1, num_channels)

        # Excite
        s = self.excite(z)
        s = s.view(-1, num_channels, 1, 1, 1)

        # Apply gate
        x = x * s

        return x

class PENetLateral(nn.Module):
    """Lateral connection layer for PENet."""
    def __init__(self, in_channels, out_channels):
        super(PENetLateral, self).__init__()

        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1, bias=False)
        self.norm = nn.GroupNorm(out_channels // 16, out_channels)
        self.relu = nn.LeakyReLU()

    def forward(self, x, x_skip):
        # Reduce number of channels in skip connection
        x_skip = self.conv(x_skip)
        x_skip = self.norm(x_skip)
        x_skip = self.relu(x_skip)

        # Add reduced feature map
        x += x_skip

        return x

class PENetDecoder(nn.Module):
    """Decoder (up-sampling layer) for PENet"""
    def __init__(self, skip_channels, in_channels, mid_channels, out_channels, kernel_size=4, stride=2):
        super(PENetDecoder, self).__init__()

        if skip_channels > 0:
            self.lateral = PENetLateral(skip_channels, in_channels)

        self.conv1 = nn.Conv3d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False)
        self.norm1 = nn.GroupNorm(mid_channels // 16, mid_channels)
        self.relu1 = nn.LeakyReLU()

        self.conv2 = nn.ConvTranspose3d(mid_channels, mid_channels, kernel_size=kernel_size, stride=stride, padding=1)
        self.norm2 = nn.GroupNorm(mid_channels // 16, mid_channels)
        self.relu2 = nn.LeakyReLU()

        self.conv3 = nn.Conv3d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.norm3 = nn.GroupNorm(out_channels // 16, out_channels)
        self.relu3 = nn.LeakyReLU()

    def forward(self, x, x_skip=None):
        if x_skip is not None:
            x = self.lateral(x, x_skip)

        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)

        return x

class PENetASPPool(nn.Module):
    """Atrous Spatial Pyramid Pooling layer.

    Based on the paper:
    "Rethinking Atrous Convolution for Semantic Image Segmentation"
    by Liang-Chieh Chen, George Papandreou, Florian Schroff, Hartwig Adam
    (https://arxiv.org/abs/1706.05587).
    """
    def __init__(self, in_channels, out_channels):
        super(PENetASPPool, self).__init__()

        self.mid_channels = out_channels // 4
        self.in_conv = nn.Sequential(nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=2, dilation=2),
                                     nn.GroupNorm(out_channels // 16, out_channels),
                                     nn.LeakyReLU(inplace=True))

        self.conv1 = nn.Conv3d(out_channels, self.mid_channels, kernel_size=1)
        self.conv2 = nn.Conv3d(out_channels, self.mid_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3 = nn.Conv3d(out_channels, self.mid_channels, kernel_size=3, padding=12, dilation=12)
        self.conv4 = nn.Sequential(nn.AdaptiveAvgPool3d(1),
                                   nn.Conv3d(out_channels, self.mid_channels, kernel_size=1))
        self.norm = nn.GroupNorm(out_channels // 16, out_channels)
        self.relu = nn.LeakyReLU(inplace=True)

        self.out_conv = nn.Sequential(nn.Conv3d(out_channels, out_channels, kernel_size=1),
                                      nn.GroupNorm(out_channels // 16, out_channels),
                                      nn.LeakyReLU(inplace=True))

    def forward(self, x):
        x = self.in_conv(x)

        # Four parallel paths with different dilation factors
        x_1 = self.conv1(x)
        x_2 = self.conv2(x)
        x_3 = self.conv3(x)
        x_4 = self.conv4(x)
        x_4 = x_4.expand(-1, -1, x_1.size(2), x_1.size(3), x_1.size(4))

        # Combine parallel pathways
        x = torch.cat((x_1, x_2, x_3, x_4), dim=1)
        x = self.norm(x)
        x = self.relu(x)

        x = self.out_conv(x)

        return x

class GAPLinear(nn.Module):
    def __init__(self, in_channels, out_channels):
        """Global average pooling (3D) followed by a linear layer.

        Args:
            in_channels: Number of input channels.
            out_channels: Number of output channels
        """
        super(GAPLinear, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(in_channels, out_channels)
        self.fc.is_output_head = True

    def forward(self, x):
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    
class PENetBottleneck(nn.Module):
    """PENet bottleneck block, similar to a pre-activation ResNeXt bottleneck block.

    Based on the paper:
    "Aggregated Residual Transformations for Deep Nerual Networks"
    by Saining Xie, Ross Girshick, Piotr Dollár, Zhuowen Tu, Kaiming He
    (https://arxiv.org/abs/1611.05431).
    """

    expansion = 2

    def __init__(self, in_channels, channels, block_idx, total_blocks, cardinality=32, stride=1):
        super(PENetBottleneck, self).__init__()
        mid_channels = cardinality * int(channels / cardinality)
        out_channels = channels * self.expansion
        self.survival_prob = self._get_survival_prob(block_idx, total_blocks)

        self.down_sample = None
        if stride != 1 or in_channels != channels * PENetBottleneck.expansion:
            self.down_sample = nn.Sequential(
                nn.Conv3d(in_channels, channels * PENetBottleneck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(channels * PENetBottleneck.expansion // 16, channels * PENetBottleneck.expansion))

        self.conv1 = nn.Conv3d(in_channels, mid_channels, kernel_size=1, bias=False)
        self.norm1 = nn.GroupNorm(mid_channels // 16, mid_channels)
        self.relu1 = nn.LeakyReLU(inplace=True)

        self.conv2 = nn.Conv3d(mid_channels, mid_channels, kernel_size=3,
                               stride=stride, padding=1, groups=cardinality, bias=False)
        self.norm2 = nn.GroupNorm(mid_channels // 16, mid_channels)
        self.relu2 = nn.LeakyReLU(inplace=True)

        self.conv3 = nn.Conv3d(mid_channels, out_channels, kernel_size=1, bias=False)
        self.norm3 = nn.GroupNorm(out_channels // 16, out_channels)
        self.norm3.is_last_norm = True
        self.relu3 = nn.LeakyReLU(inplace=True)

        self.se_block = SEBlock(out_channels, reduction=16)

    @staticmethod
    def _get_survival_prob(block_idx, total_blocks, p_final=0.5):
        """Get survival probability for stochastic depth. Uses linearly decreasing
        survival probability as described in "Deep Networks with Stochastic Depth".

        Args:
            block_idx: Index of residual block within entire network.
            total_blocks: Total number of residual blocks in entire network.
            p_final: Survival probability of the final layer.
        """
        return 1. - block_idx / total_blocks * (1. - p_final)

    def forward(self, x):
        x_skip = x if self.down_sample is None else self.down_sample(x)

        # Stochastic depth dropout
        if self.training and random.random() > self.survival_prob:
            return x_skip

        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.norm3(x)

        x = self.se_block(x)
        x += x_skip

        x = self.relu3(x)

        return x
    
class PENetEncoder(nn.Module):
    def __init__(self, in_channels, channels, num_blocks, cardinality, block_idx, total_blocks, stride=1):
        super(PENetEncoder, self).__init__()

        # Get PENet blocks
        penet_blocks = [PENetBottleneck(in_channels, channels, block_idx, total_blocks, cardinality, stride)]

        for i in range(1, num_blocks):
            penet_blocks += [PENetBottleneck(channels * PENetBottleneck.expansion, channels,
                                           block_idx + i, total_blocks, cardinality)]
        self.penet_blocks = nn.Sequential(*penet_blocks)

    def forward(self, x):
        x = self.penet_blocks(x)

        return x

class PENetClassifier(nn.Module):
    """PENet stripped down for classification.

    The idea is to pre-train this network, then use the pre-trained
    weights for the encoder in a full PENet.
    """

    def __init__(self, model_depth, cardinality=32, num_channels=3, num_classes=600, init_method=None, **kwargs):
        super(PENetClassifier, self).__init__()

        self.in_channels = 64
        self.model_depth = model_depth
        self.cardinality = cardinality
        self.num_channels = num_channels
        self.num_classes = num_classes

        self.in_conv = nn.Sequential(nn.Conv3d(self.num_channels, self.in_channels, kernel_size=7,
                                               stride=(1, 2, 2), padding=(3, 3, 3), bias=False),
                                     nn.GroupNorm(self.in_channels // 16, self.in_channels),
                                     nn.LeakyReLU(inplace=True))
        self.max_pool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)

        # Encoders
        if model_depth != 50:
            raise ValueError('Unsupported model depth: {}'.format(model_depth))
        encoder_config = [3, 4, 6, 3]
        total_blocks = sum(encoder_config)
        block_idx = 0

        self.encoders = nn.ModuleList()
        for i, num_blocks in enumerate(encoder_config):
            out_channels = 2 ** i * 128
            stride = 1 if i == 0 else 2
            encoder = PENetEncoder(self.in_channels, out_channels, num_blocks, self.cardinality,
                                  block_idx, total_blocks, stride=stride)
            self.encoders.append(encoder)
            self.in_channels = out_channels * PENetBottleneck.expansion
            block_idx += num_blocks

        self.classifier = GAPLinear(self.in_channels, num_classes)

        if init_method is not None:
            self._initialize_weights(init_method, focal_pi=0.01)

    def _initialize_weights(self, init_method, gain=0.2, focal_pi=None):
        """Initialize all weights in the network."""
        for m in self.modules():
            if isinstance(m, nn.Conv3d) or isinstance(m, nn.ConvTranspose3d) or isinstance(m, nn.Linear):
                if init_method == 'normal':
                    nn.init.normal_(m.weight, mean=0, std=gain)
                elif init_method == 'xavier':
                    nn.init.xavier_normal_(m.weight, gain=gain)
                elif init_method == 'kaiming':
                    nn.init.kaiming_normal_(m.weight)
                else:
                    raise NotImplementedError('Invalid initialization method: {}'.format(self.init_method))
                if hasattr(m, 'bias') and m.bias is not None:
                    if focal_pi is not None and hasattr(m, 'is_output_head') and m.is_output_head:
                        # Focal loss prior (~0.01 prob for positive, see RetinaNet Section 4.1)
                        nn.init.constant_(m.bias, -math.log((1 - focal_pi) / focal_pi))
                    else:
                        nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.GroupNorm) and m.affine:
                # Gamma for last GroupNorm in each residual block gets set to 0
                init_gamma = 0 if hasattr(m, 'is_last_norm') and m.is_last_norm else 1
                nn.init.constant_(m.weight, init_gamma)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):

        # Expand input (allows pre-training on RGB videos, fine-tuning on Hounsfield Units)
        if x.size(1) < self.num_channels:
            x = x.expand(-1, self.num_channels // x.size(1), -1, -1, -1)

        x = self.in_conv(x)
        x = self.max_pool(x)

        # Encoders
        for encoder in self.encoders:
            x = encoder(x)

        # Classifier
        x = self.classifier(x)

        return x

    def args_dict(self):
        """Get a dictionary of args that can be used to reconstruct this architecture.
        To use the returned dict, initialize the model with `PENet(**model_args)`.
        """
        model_args = {
            'model_depth': self.model_depth,
            'cardinality': self.cardinality,
            'num_classes': self.num_classes,
            'num_channels': self.num_channels
        }

        return model_args

    def load_pretrained(self, ckpt_path, gpu_ids):
        """Load parameters from a pre-trained PENetClassifier from checkpoint at ckpt_path.
        Args:
            ckpt_path: Path to checkpoint for PENetClassifier.
        Adapted from:
            https://discuss.pytorch.org/t/how-to-load-part-of-pre-trained-model/1113/2
        """
        device = 'cuda:{}'.format(gpu_ids[0]) if len(gpu_ids) > 0 else 'cpu'
        pretrained_dict = torch.load(ckpt_path, map_location=device)['model_state']
        model_dict = self.state_dict()

        # Filter out unnecessary keys
        pretrained_dict = {k[len('module.'):]: v for k, v in pretrained_dict.items()}
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}

        # Overwrite entries in the existing state dict
        model_dict.update(pretrained_dict)

        # Load the new state dict
        self.load_state_dict(model_dict)

    def fine_tuning_parameters(self, fine_tuning_boundary, fine_tuning_lr=0.0):
        """Get parameters for fine-tuning the model.
        Args:
            fine_tuning_boundary: Name of first layer after the fine-tuning layers.
            fine_tuning_lr: Learning rate to apply to fine-tuning layers (all layers before `boundary_layer`).
        Returns:
            List of dicts that can be passed to an optimizer.
        """

        def gen_params(boundary_layer_name, fine_tuning):
            """Generate parameters, if fine_tuning generate the params before boundary_layer_name.
            If unfrozen, generate the params at boundary_layer_name and beyond."""
            saw_boundary_layer = False
            for name, param in self.named_parameters():
                if name.startswith(boundary_layer_name):
                    saw_boundary_layer = True

                if saw_boundary_layer and fine_tuning:
                    return
                elif not saw_boundary_layer and not fine_tuning:
                    continue
                else:
                    yield param

        # Fine-tune the network's layers from encoder.2 onwards
        optimizer_parameters = [{'params': gen_params(fine_tuning_boundary, fine_tuning=True), 'lr': fine_tuning_lr},
                                {'params': gen_params(fine_tuning_boundary, fine_tuning=False)}]

        # Debugging info
        util.print_err('Number of fine-tuning layers: {}'
                       .format(sum(1 for _ in gen_params(fine_tuning_boundary, fine_tuning=True))))
        util.print_err('Number of regular layers: {}'
                       .format(sum(1 for _ in gen_params(fine_tuning_boundary, fine_tuning=False))))

        return optimizer_parameters
    
def peNet(num_classes):
    net = PENetClassifier(50, num_classes = 1)
    net = nn.DataParallel(net)
    net.load_state_dict(torch.load('../input/mymodels/penet.pth'))
    net.module.classifier.fc = nn.Linear(2048, num_classes)
    return(net)

In [ ]:
def fill_eval_tab(model_name, eval_tab, tij):
    dataset = PE_dataset_pipeline(eval_tab, tij)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = 4, num_workers = 2)
    #load model
    if model_name == 'chronic_net':
        net = peNet(3)
        net.load_state_dict(torch.load('../input/mymodels/best_chronic.pth'))
        net = net.eval().cuda()
    else:
        net = peNet(2)
        net = models[model_name](net)
    #fill table
    for i, (inpt, index) in tqdm(enumerate(dataloader)):
        outpt = net(inpt)
        for j, idx in enumerate(index):
            if model_name == 'chronic_net':
                eval_tab['negative_chronic'][idx.item()] = outpt[j][0].item()
                eval_tab['chronic'][idx.item()] = outpt[j][1].item()
                eval_tab['acute'][idx.item()] = outpt[j][2].item()
            else:
                eval_tab[C[model_name][0]][idx.item()] = outpt[j][0].item()
                eval_tab[C[model_name][1]][idx.item()] = outpt[j][1].item()
        outpt = None

In [ ]:
def fill_eval_tab2(eval_tab, tij):
    dataset = PE_dataset_pipeline(eval_tab, tij)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = 4, num_workers = 2)
    
    #fill table
    for i, (inpt, index) in tqdm(enumerate(dataloader)):
        for model_name in ['big_detect_net', 'right_det_net', 'left_net', 'central_net', 'chronic_net']:
            #load model
            if model_name == 'chronic_net':
                net = peNet(3)
                net.load_state_dict(torch.load('../input/mymodels/best_chronic.pth'))
                net = net.eval().cuda()
            else:
                net = peNet(2)
                net = models[model_name](net)
            outpt = net(inpt)
            for j, idx in enumerate(index):
                if model_name == 'chronic_net':
                    eval_tab['negative_chronic'][idx.item()] = outpt[j][0].item()
                    eval_tab['chronic'][idx.item()] = outpt[j][1].item()
                    eval_tab['acute'][idx.item()] = outpt[j][2].item()
                else:
                    eval_tab[C[model_name][0]][idx.item()] = outpt[j][0].item()
                    eval_tab[C[model_name][1]][idx.item()] = outpt[j][1].item()
            outpt = None
            net = None

In [ ]:
fill_eval_tab2(eval_tab, tij)

In [ ]:
#fill_eval_tab('chronic_net', eval_tab, tij)
#fill_eval_tab('big_detect_net', eval_tab, tij)
#fill_eval_tab('right_det_net', eval_tab, tij)
#fill_eval_tab('left_net', eval_tab, tij)
#fill_eval_tab('central_net', eval_tab, tij)

In [ ]:
class PE_dataset_pipeline(torch.utils.data.Dataset):
    def __init__(self, Table, eval_tab, window = 48):
        self.slices = list(eval_tab['Exam'].unique())
        self.window = window
        self.Table = Table
        self.eval_tab = eval_tab
            
    def __getitem__(self, index):
        mini_batch = self.eval_tab[self.eval_tab.Exam == self.slices[index]].reset_index().drop(['index'], axis = 1)
        Study = mini_batch['Exam'][0]
        temp = self.Table[self.Table.StudyInstanceUID == Study].reset_index().drop(['index'], axis = 1)
        
        #input construction
        Input = torch.zeros(7*4)
        Softner = nn.Softmax(dim = 0)
        for i in mini_batch.index:
            Input[i*7] = Softner(torch.Tensor([mini_batch['negative_detect'][i], mini_batch['positive_detect'][i]]))[1]
            Input[i*7 + 1] = float(mini_batch['right_detect'][i])
            Input[i*7 + 2] = float(mini_batch['right'][i])
            Input[i*7 + 3] = Softner(torch.Tensor([mini_batch['negative_left'][i], mini_batch['positive_left'][i]]))[1]
            Input[i*7 + 4] = Softner(torch.Tensor([mini_batch['negative_central'][i], mini_batch['positive_central'][i]]))[1]
            Input[i*7 + 5] = Softner(torch.Tensor([mini_batch['negative_chronic'][i], mini_batch['chronic'][i], mini_batch['acute'][i]]))[1]
            Input[i*7 + 6] = Softner(torch.Tensor([mini_batch['negative_chronic'][i], mini_batch['chronic'][i], mini_batch['acute'][i]]))[2]
        return(Input, self.Table[self.Table.StudyInstanceUID == Study].index[0])
    def __len__(self):
        return(len(self.slices))

In [ ]:
class IntegNet(nn.Module):
    def __init__(self):
        super(IntegNet, self).__init__()
        self.dropout_rate = 0.5878
        fc2_input_dim = 804
        fc3_input_dim = 670
        self.fc1 = nn.Linear(4*7, fc2_input_dim)
        self.fc2 = nn.Linear(fc2_input_dim, fc3_input_dim)
        self.fco = nn.Linear(fc3_input_dim, fc3_input_dim)
        self.fc3 = nn.Linear(fc3_input_dim, 9)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = F.relu(self.fco(x))
        x = F.dropout(x, p = self.dropout_rate, training=self.training)
        x = self.fc3(x)
        return x.clamp(min = 0, max = 1)

# Generate the model.
net = IntegNet()
net.load_state_dict(torch.load('../input/mymodels/best_Integnet.pth'))

In [ ]:
class pe_dataset(torch.utils.data.Dataset):
    def __init__(self, Table, augment = True):
        self.images = list(Table.index)
        self.Table = Table
        self.augmented = augment
    def __getitem__(self, index):
        image = read_image([self.Table['path'][self.images[index]]])
        return(image, self.images[index])
    def __len__(self):
        return(len(self.images))

In [ ]:
def Resnet50(num_classes, pretrained = True):
    net = torchvision.models.resnet50(pretrained=pretrained, progress=True)
    out_size = net.fc.in_features
    net.fc = nn.Linear(out_size, num_classes)
    return(net)
def Resnet152(num_classes, pretrained = True):
    net = torchvision.models.resnet152(pretrained=False, progress=True)
    out_size = net.fc.in_features
    net.fc = nn.Linear(out_size, num_classes)
    return(net)

In [ ]:
def Submit(Exam_net, detect_net, eval_tab, tij, df):
    dataset = PE_dataset_pipeline(tij, eval_tab)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = 1, num_workers = 2)
    
    sub = pd.read_csv('./submission.csv')
    Elements = set(list(sub['id'][i].split('_')[0] for i in sub.index))
    
    Exam_net = Exam_net.eval().cuda()
    for i, (Input, idx) in tqdm(enumerate(dataloader)):
        Exam = tij['StudyInstanceUID'][idx[0].item()]
        if Exam in Elements:
            continue
        outpt = Exam_net(Input[0].cuda())
        temp = pd.DataFrame([[Exam+'_indeterminate', outpt[0].item()],
                             [Exam+'_negative_exam_for_pe', outpt[1].item()],
                             [Exam+'_central_pe', outpt[2].item()],
                             [Exam+'_rightsided_pe', outpt[2].item()],
                             [Exam+'_chronic_pe', outpt[3].item()],
                             [Exam+'_acute_and_chronic_pe', outpt[4].item()],
                             [Exam+'_rv_lv_ratio_gte_1', outpt[5].item()],
                             [Exam+'_rv_lv_ratio_lt_1', outpt[6].item()]], columns = sub.columns)
        sub = pd.concat([sub, temp])
    dataset = pe_dataset(df)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = 15, num_workers = 2)
    detect_net = detect_net.cuda().eval()
    Softner = nn.Softmax()
    for i, (Input, idx) in tqdm(enumerate(dataloader)):
        outpt = detect_net(Input.cuda())
        #print(output)
        for i, index in enumerate(idx):
            SOP = df['SOPInstanceUID'][index.item()]
            if SOP in Elements:
                continue
            pred = Softner(outpt[i])[1].item()
            temp = pd.DataFrame([[SOP, pred]], columns = sub.columns)
            sub = pd.concat([sub, temp])
    sub = sub[['id', 'label']]
    sub.to_csv('./submission2.csv')

In [ ]:
#tij = pd.read_csv('../input/submission/test_exams.csv')
#eval_tab = pd.read_csv('../input/submission/test_eval_tab.csv')
detect_net = Resnet152(2)
detect_net.load_state_dict(torch.load('../input/mymodels/best_detection.pth'))
Submit(net, detect_net, eval_tab, tij, df)